# ERA5 HI, WBGT & Air Temps
Cascade Tuholske, Oct 2024

This is a notebook to generate daily maximum heat index, maximum indoor/shared wet blube globe temperature and daily minimum/maximum air temperature using ERA5 Land climate reanalysis. The data can be downloaed from [ECWMF](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land?tab=download).

Here we are using hour dew point temperature, 2 meter air temperature, and surface pressure for July 2020. We have selected a spatial extent covering Ghana. The file is downloaded in a `NetCDF` format so that we can use `Xarray` to process the data.

While there are many heat stress metrics, we are going to use heat index and an approximated shade wet bulb globe temperature. You can read about the conversion of HI to WBGT [here](https://www.tandfonline.com/doi/abs/10.1080/15459624.2014.989365?casa_token=sExUV3cSMu0AAAAA:xGVypJnWLDwg4hvnAoAET8Hvn0_v_dca-IojLQ1jyuqrekH9_QsO6BW1djKQY9mYLgRX-aI5Rwxmcw).  

Below is a referesher on `Xarray` data structures:

#### Understanding Xarray's Data Structures

Xarray introduces two main data structures: `DataArray` and `Dataset`. These structures are built on top of NumPy and pandas. We recommend you read about [`Xarray` data structures](https://docs.xarray.dev/en/stable/user-guide/data-structures.html) because the terminology can be a bit confusing

A `DataArray` is Xarray’s implementation of a labeled, multi-dimensional array. 

- Values: a numpy.ndarray or numpy-like array holding the array’s values
- Dimensions: These are names for each axis of the array. For instance, in climate data, common dimensions include 'time', 'latitude', and 'longitude'.
- Coordinates: These are arrays that label each point, such as specific dates for the 'time' dimension or longitude and latitude values for geographical dimensions. Coordinates enable powerful and intuitive data selections.
- Attributes: These are metadata for the array, used to store arbitrary metadata such as the description of the dataset, units of measurement, or other relevant information.

A `Dataset` is a dictionary-like container that holds multiple variables (arrays) that potentially share the same coordinates. It is analogous to a pandas DataFrame but can handle multi-dimensional data. Each variable in a Dataset is itself a DataArray.

![ntl](./assets/xarray.png) <br>

Basically, `Xarray` allows you to create labeled n-dimentional numpy arrays. So you can label your datasets (temperature, precipitation, etc.) and your dimentions (time, lat/long, etc.) to easily subset the data to run analysis. For example, you could say, what is the average heat index in Accra based on Lat/Long with just a few lines of code. **Note** We will use the package [`rioxarray`](https://corteva.github.io/rioxarray/html/rioxarray.html) too to load in the GeoTiff files. More on this later.   

### Let's get started!
<hr style="border-top: 1px solid gray; margin-top: 24px; margin-bottom: 1px"></hr>

<div class="run">
    ▶️ <b> Run the cells below. </b>
</div>

In [6]:
# Dependencies 
import os
import glob 
import xarray as xr
import dask
import rioxarray as rio
import pandas as pd
import numpy as np
import rasterio
import ClimFuncs as cf
from atmos import moisture
from atmos import thermo
from atmos.moisture import relative_humidity_from_dewpoint_temperature as rh
import matplotlib.pyplot as plt

### Open the NetCDF File as an Xarray Dataset

In [7]:
fn = os.path.join('./data/ERA5-2020-07-td-t-p.nc')
ds = xr.open_dataset(fn)

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [ ]:
ds

### Check the data
Notice that the temperature data is in Kelvin ...

In [ ]:
arr = ds.d2m.isel(valid_time = 1)

In [ ]:
plt.imshow(arr)
plt.title('2m Dewpoint Temperature July 1, 2020 00h')
plt.colorbar()

In [ ]:
arr = ds.t2m.isel(valid_time = 1)

In [ ]:
plt.imshow(arr)
plt.title('2m Air Temperature July 1, 2020 00h')
plt.colorbar()

# Estimate Relative Humidity 

In [ ]:
ds['rh'] = xr.apply_ufunc(rh, ds.t2m, ds.d2m)

# Estimate Heat Index & WBGT

In [ ]:
def make_hi_wbgt(ds):
    """
    Calculate heat index (HI) and wet-bulb globe temperature (WBGT) from a dataset containing temperature and relative humidity.

    This function takes a dataset (`ds`) with temperature in Kelvin and relative humidity (RH), converts temperature 
    to Celsius and Fahrenheit, computes the heat index in Fahrenheit, converts it back to Celsius, and calculates the 
    wet-bulb globe temperature. The computed values are added as new variables to the dataset.

    Parameters
    ----------
    ds : xarray.Dataset
        An xarray dataset containing at least two variables:
        - `t2m` : Temperature in Kelvin.
        - `rh`  : Relative humidity as a percentage.

    Returns
    -------
    xarray.Dataset
        The input dataset with added variables:
        - `t2m_c` : Temperature in Celsius.
        - `t2m_f` : Temperature in Fahrenheit.
        - `hi`    : Heat index in Celsius.
        - `wbgt`  : Wet-bulb globe temperature in Celsius.

    Notes
    -----
    The function assumes that:
    - Temperature in Kelvin (`t2m`) is converted to Celsius and Fahrenheit.
    - Heat index (`hi`) is calculated in Fahrenheit and then converted back to Celsius.
    - Wet-bulb globe temperature (`wbgt`) is derived from the heat index.
    
    Requires the `cf` module for temperature and heat index conversions.
    """
    ds['t2m_c'] = ds.t2m - 273.15
    ds['t2m_f'] = xr.apply_ufunc(cf.C_to_F, ds.t2m_c)
    ds['hi'] = cf.heatindex(ds.t2m_f, ds.rh, 'F', 'F')
    ds['wbgt'] = xr.apply_ufunc(cf.hi_to_wbgt, ds.hi)
    ds['hi'] = xr.apply_ufunc(cf.F_to_C, ds.hi)
    
    return ds

In [ ]:
ds = make_hi_wbgt(ds)

In [ ]:
ds.hi.max(dim = 'valid_time')

### Check the data
Let's look at the hi and wbgt data

In [ ]:
# Define subplots with smaller size
fig, axs = plt.subplots(3, 2, figsize=(15, 15))  # 2 row, 2 columns

arr = ds.hi.isel(valid_time = 12)
p1 = axs[0][0].imshow(arr)
cbar = fig.colorbar(p1, ax=axs[0][0], orientation='vertical', fraction=0.02, pad=0.04)
axs[0][0].set_title('Heat Index July 1, 2020 00h')

arr = ds.wbgt.isel(valid_time = 12)
p1 = axs[0][1].imshow(arr)
cbar = fig.colorbar(p1, ax=axs[0][1], orientation='vertical', fraction=0.02, pad=0.04)
axs[0][1].set_title('WBGT July 1, 2020 00h')

arr = ds.t2m_c.isel(valid_time = 12)
p1 = axs[1][0].imshow(arr)
cbar = fig.colorbar(p1, ax=axs[1][0], orientation='vertical', fraction=0.02, pad=0.04)
axs[1][0].set_title('T2m (°C) July 1, 2020 00h')

arr = ds.t2m_f.isel(valid_time = 12)
p1 = axs[1][1].imshow(arr)
cbar = fig.colorbar(p1, ax=axs[1][1], orientation='vertical', fraction=0.02, pad=0.04)
axs[1][1].set_title('T2m (°F) July 1, 2020 00h')

arr = ds.rh.isel(valid_time = 12)
p1 = axs[2][0].imshow(arr)
cbar = fig.colorbar(p1, ax=axs[2][0], orientation='vertical', fraction=0.02, pad=0.04)
axs[2][0].set_title('RH July 1, 2020 00h')

arr = ds.d2m.isel(valid_time = 12)
p1 = axs[2][1].imshow(arr)
cbar = fig.colorbar(p1, ax=axs[2][1], orientation='vertical', fraction=0.02, pad=0.04)
axs[2][1].set_title('DewPoint Temp (°c) July 1, 2020 00h')

In [ ]:
arr = ds.hi.isel(valid_time = 1)

In [ ]:
plt.imshow(arr)
plt.title('Heat Index July 1, 2020 00h')
plt.colorbar()

In [ ]:
arr = ds.wbgt.isel(valid_time = 1)

In [ ]:
plt.imshow(arr)
plt.title('WBGT July 1, 2020 00h')
plt.colorbar()

# Daily Max
Now, let's select the daily maximum heat index, wet bulb glob temperature, min/max air temperature.

In [ ]:
# Select the daily values
tmin = ds.t2m_c.resample(valid_time='1D').min(dim = 'valid_time')
tmax = ds.t2m_c.resample(valid_time='1D').max(dim = 'valid_time')
himax = ds.hi.resample(valid_time='1D').max(dim = 'valid_time')
wbgtmax = ds.wbgt.resample(valid_time='1D').max(dim = 'valid_time')

In [ ]:
# rename tmin and tmax 
tmin = tmin.rename('tmin')
tmax = tmax.rename('tmax')

In [ ]:
# take a look
tmin

In [ ]:
# Concatenate the daily min/max values into a single dataset 
ds_out = xr.merge([tmin, tmax, himax, wbgtmax])

In [ ]:
# Check out ds_out
ds_out

# Save out the daily min/max data

In [ ]:
fn_out = os.path.join('./data/ERA5-2020-07-dailyheat.nc')

In [ ]:
# Save the dataset to a NetCDF file
ds_out.to_netcdf(fn_out)